<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="7"> <b> GEOS 657: Microwave Remote Sensing<b> </font>

<font size="5"> <b>Lab 4: Exploring SAR Data and SAR Time Series Analysis using Jupyter Notebooks <font color='rgba(200,0,0,0.2)'> -- [20 Points] </font> </b> </font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks & Josef Kellndorfer, <a href="http://earthbigdata.com/" target="_blank">Earth Big Data, LLC</a> </b> <br>
<img src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /><font color='rgba(200,0,0,0.2)'> <b>Due Date: </b> March 05, 2019 </font>
</font>

<font size="3"> This Lab is part of the UAF course <a href="https://radar.community.uaf.edu/" target="_blank">GEOS 657: Microwave Remote Sensing</a>. It is introducing you to the analysis of deep multi-temporal SAR image data stacks in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>In this chapter we introduce the following data analysis concepts:</b>

- How to load time series stacks into Jupyter Notebooks and how to explore image content using basic functions such as mean value calculation and histogram analysis.
- How to apply calibration constants to covert initial digital number (DN) data into calibrated radar cross section information.
- How to subset images create time series information of calibrated SAR amplitude values.
- How to explore the time-series information in SAR data stacks for environmental analysis.
</font>

<font size="4"> <font color='rgba(200,0,0,0.2)'> <b>THIS NOTEBOOK INCLUDES FOUR HOMEWORK ASSIGNMENTS.</b></font> Complete all four assignments to achieve full score. </font> <br>
<font size="3"> To submit your homework, please download your Jupyter Notebook from the server both asf PDF (*.pdf) and Notebook file (*.ipynb) and submit them as a ZIP bundle via Blackboard or email (to fjmeyer@alaska.edu). To download, please select the following options in the main menu of the notebook interface:

<ol type="1">
  <li><font color='rgba(200,0,0,0.2)'> <b> Save your notebook with all of its content</b></font> by selecting <i> File / Save and Checkpoint </i> </li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in Notebook format</b></font>, click on <i>File / Download as / Notebook (.ipynb)</i>  <font color='gray'>--- Downloading your file may take a bit as the Notebook will be about 100MB in size</font></li>
  <li><font color='rgba(200,0,0,0.2)'> <b>To export in PDF format</b></font>, click on <i>File / Download as / PDF vs LaTeX (.pdf) </i></li>
</ol>

Contact me at fjmeyer@alaska.edu should you run into any problems.
</font>

</font>

<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3"> The first step of this lab exercises on SAR image time series analysis is the import of necessary python libraries into your Jupyter Notebook. See the code cell below for information on which libraries are needed. Information on these libraries is provided in the instructions to the previous lab of this course (Lab 3). 

</font>

In [ ]:
import pandas as pd
import gdal
import numpy as np
import time,os, glob

# Setup plotting inside the notebook
%matplotlib inline
import matplotlib.pylab as plt

<hr>
<font face="Calibri">

<font size="5"> <b> 1. Load Data Stack for this Lab </b> </font> <img src="NotebookAddons/Nepalclimate.jpeg" width="400" align="right" /> 

<font size="3"> This Lab will be using a 70-image deep L-band SAR data stack over Nepal for a first experience with time series processing. The L-band data were acquired by the ALOS PALSAR sensor and are available to us through the services of the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility</a>. 

Nepal is an interesting site for this lab due to the significant seasonality of precipitation that is characteristic for this region. Nepal is said to have five seasons: spring, summer, monsoon, autumn and winter. Precipitation is low in the winter (November - March) and peaks dramatically in the summer, with top rain rates in July, August, and September (see figure to the right). As SAR is sensitive to changes in soil moisture, these weather patterns have a noticeable impact on the Radar Cross Section ($\sigma$) time series information. 

We will analyze the variation of $\sigma$ values over time and will interpret them in the context of rainfall rates in the imaged area. 

In this case, we will <b>retrieve the relevant data</b> from an <a href="https://aws.amazon.com/" target="_blank">Amazon Web Service (AWS)</a> cloud storage bucket, <b>using the following command</b>:</font> 

</font>

In [ ]:
!aws s3 cp s3://asf-jupyter-data/time_series.zip time_series.zip

<font face="Calibri" size="3"> Now, let's <b>unzip the file and clean up after ourselves:</b> </font>

In [ ]:
!unzip time_series.zip
!rm time_series.zip

<font face="Calibri" size="3"> The following lines set variables that capture path variables needed for data processing. <b>We define variables for the main data directory as well as the main variables containing data and image information:</b> </font>

In [ ]:
datadirectory='/home/jovyan/notebooks/ASF/GEOS_657_Labs/time_series/S32644X696260Y3052060sS1-EBD'
datefile='S32644X696260Y3052060sS1_D_vv_0092_mtfil.dates'
imagefile='S32644X696260Y3052060sS1_D_vv_0092_mtfil.vrt'
imagefile_cross='S32644X696260Y3052060sS1_D_vh_0092_mtfil.vrt'

In [ ]:
os.getcwd()  # Uncomment this line to display the present working directory

<br>
<hr>
<font face="Calibri" size="5"> <b> 2. Switch to the Data Directory: </b> 

<font size="3"> We now switch into the data directory and briefly verify that we ended up in the correct directory.</font>

</font> 

In [ ]:
os.chdir(datadirectory)

In [ ]:
os.getcwd()  # Uncomment this line to display the present working directory

In [ ]:
# glob.glob("*.vrt")   # Uncomment this line to see a List of the files 

<br>
<hr>
<font face="Calibri" size="5"> <b> 3. Assess Image Acquisition Dates </b> </font> 

<font face="Calibri" size="3"> Before we start analyzing the available image data, we want to examine the content of our data stack. <b>First, we read the image acquisition dates for all files in the time series and create a *pandas* date index.</b> </font>

In [ ]:
dates=open(datefile).readlines()
tindex=pd.DatetimeIndex(dates)

<font face="Calibri" size="3"> From the date index we <b>make and print a lookup table for band numbers and dates:</b> </font>

In [ ]:
j=1
print('Bands and dates for',imagefile)
for i in tindex:
    print("{:4d} {}".format(j, i.date()),end=' ')
    j+=1
    if j%5==1: print()

<br>
<hr>
<font face="Calibri" size="5"> <b> 4. Explore the Available Image Data </b> </font> 

<font face="Calibri" size="3"> To **open** an image file and make it readable we use the gdal.Open() function. This generates an image handle that can be used for further interactions with the file: </font>

In [ ]:
img=gdal.Open(imagefile)

<font face="Calibri" size="3"> <b>To explore the image, e.g. number of bands, pixels, lines</b> you can use several functions associated with the openend image object, e.g.: </font>

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

<br>
<font face="Calibri" size="4"> <b> 4.1 Reading Data from an Image Band </b> </font> 

<font face="Calibri" size="3"> <b>To access any band in the image</b>, use the *img.GetRasterBand(x)* function. E.g. to access the first band x=1, the last band: x=70. </font>

In [ ]:
band=img.GetRasterBand(70)

<font face="Calibri" size="3"> Once a band is seleted, several functions associated with the band are available for further processing, e.g., <i>band.ReadAsArray(xoff=0,yoff=0,xsize=None,ysize=None)</i>

So, <b>to read the entire raster layer for the band:</b> </font>

In [ ]:
raster=band.ReadAsArray()

<br>
<font face="Calibri" size="4"> <b> 4.2 Extracting Subsets from a Larger Image Frame </b> </font>

<font face="Calibri" size="3"> Because of the potentially large data volume when dealing with time series data stacks, it may be prudent to read only a subset of data. 

With the <i>gdal .ReadAsArray()</i> function, subsets can be requested by defining pixel offsets and subset size:

**img.ReadAsArray(xoff=0, yoff=0, xsize=None, ysize=None)**

- <i>xoff, yoff</i> are the offsets from the upper left corner in pixel/line coordinates. 
- <i>xsize, ysize</i> specify the size of the subset in x-direction (left to right) and y-direction (top to bottom).

E.g., to read only a **subset of 5x5 pixels with an offset of 5 pixels and 20 lines**: </font>

In [ ]:
raster_sub=band.ReadAsArray(5,20,50,50)

<font face="Calibri" size="3"> The result is a two dimensional numpy array in the datatpye the data were stored in. **We can inspect these data in python by typing the array name on the commandline**: </font>

In [ ]:
raster_sub

<br>
<font face="Calibri" size="4"> <b> 4.3 Displaying Bands in the Time Series of SAR Data </b> </font>

<font face="Calibri" size="3"> From the lookup table we know that bands 20 and 27 in the Nepal data stack are from late February and late August. **Let's take look at these images**. </font>

In [ ]:
raster_1 = img.GetRasterBand(20).ReadAsArray()
raster_2 = img.GetRasterBand(27).ReadAsArray()

<br>
<font face="Calibri" size="3"> <b> <i>4.4.1 Plotting in Python to Visualize the Image Bands </i></b> </font>

<font face="Calibri" size="3"> Matplotlib's plotting functions allow for powerful options to display imagery. We are following some standard approaches for setting up figures.
First we are looking at a **raster band** and it's associated **histogram**. </font>

In [ ]:
fig = plt.figure(figsize=(16,8)) # Initialize figure with a size
ax1 = fig.add_subplot(121)  # 121 determines: 1 row, 2 plots, first plot
ax2 = fig.add_subplot(122)  # 122 determines: 1 row, 2 plots, second plot

# First plot: Image
bandnbr=20
ax1.imshow(raster_1,cmap='gray',vmin=2000,vmax=10000)
ax1.set_title('Image Band {} {}'.format(bandnbr,
                                    tindex[bandnbr-1].date()))

# Second plot: Histogram
# IMPORTANT: To get a histogram, we first need to *flatten* 
# the two-dimensional image into a one-dimensional vector.
h = ax2.hist(raster_1.flatten(),bins=200,range=(0,10000))
ax2.xaxis.set_label_text('Amplitude (Uncalibrated DN Values)')
ax2.set_title('Histogram Band {} {}'.format(bandnbr,
                                    tindex[bandnbr-1].date()))

<br>
<font face="Calibri" size="3"> <b> <i>4.3.2 Writing a Plotting Function for the Task </i></b> </font>

<font face="Calibri" size="3"> In the following, we use the plotting commands from the previous code cell to **define** a function named *showImage*. Several parameters can be passed to the function, some with default values listed at the end:
- raster = a numpy two dimensional array 
- tindex = a panda index array for dates
- bandnbr = the band number the corresponds to the raster 
- vmin = minimim value to display 
- vmax = maximum value to display

Note: By default, data will be linearly stretched between vmin and vmax. </font>

In [ ]:
def showImage(raster,tindex,bandnbr,vmin=None,vmax=None):
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.imshow(raster,cmap='gray',vmin=vmin,vmax=vmax)
    ax1.set_title('Image Band {} {}'.format(bandnbr,
                                tindex[bandnbr-1].date()))
    vmin=np.percentile(raster,2) if vmin==None else vmin
    vmax=np.percentile(raster,98) if vmax==None else vmax
    ax1.xaxis.set_label_text(
        'Linear stretch Min={} Max={}'.format(vmin,vmax))
    
    
    h = ax2.hist(raster.flatten(),bins=100,range=(0,11000))
    ax2.xaxis.set_label_text('Amplitude (Uncalibrated DN Values)')
    ax2.set_title('Histogram Band {} {}'.format(bandnbr,
                                tindex[bandnbr-1].date()))

<br>
<hr>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #1</u>:  </font> Read Two Different Bands and Display them Using <i>showImage()</i> </b> <font color='rgba(200,0,0,0.2)'> -- [4 Points] </font> </font>

<font face="Calibri" size="3"> <b>This assignment includes four steps:</b> (1) Use the acquisition date information to select two new bands to display; (1) use <i>bandnbr</i> as the variable name to identify your bands of interest in the data stack; (3) use the already open image handle <i>img</i> to obtain the raster data from a band; (4) visualize the band using *showImage()*. </font>
</div>
<hr>

In [ ]:
# ENTER YOUR CODE FOR LOADING AND DISPLAYING YOUR FIRST IMAGE HERE:

<hr>

In [ ]:
# ENTER YOUR CODE FOR LOADING AND DISPLAYING YOUR SECOND IMAGE HERE:

<hr>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #2</u>:  </font> Analyze Pairs of Image Bands by Displaying them Side-by-Side </b> <font color='rgba(200,0,0,0.2)'> -- [6 Points] </font> </font>

<font face="Calibri" size="3"> <u>For this exercise please complete the following activities</u>:
<ol type="1">
    <li><b>Write a function to display two different image bands side-by-side</b>. Make sure that the sub-figure titles disclose the acquisition dates of the images and that the images are scaled the same to the identical DN range (by specifying (vmin,vmax)). You can steal code elements from function <i>showImage()</i> for this exercise. <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
    <br>
    <li><b>Create a figure that compares two images</b> from the <b>winter vs. the summer</b> season.<font color='rgba(200,0,0,0.2)'> -- [1 Point] </font></li>
    <br>
    <li><b>Describe the differences you see between these images</b> and make a suggestion what may have caused these differences.<font color='rgba(200,0,0,0.2)'> -- [1 Points] </font></li>   
    <br>
    <li><b>Create a figure that compares two images from different years</b> (e.g., 2014 vs 2017).<font color='rgba(200,0,0,0.2)'> -- [1 Point] </font></li>
    <br>
    <li><b>Describe the differences you see between these images</b> and make a suggestion what may have caused these differences.<font color='rgba(200,0,0,0.2)'> -- [1 Points] </font></li> 
</ol> 
</font>
</div>
<hr>

In [ ]:
# Step 1: Write function for side-by-side image display
# ENTER YOUR CODE HERE

<hr>

In [ ]:
# Step 2: Pick images from winter and summer and display them side-by-side
# ENTER YOUR CODE HERE

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Step 3: </font>Describe the differences you see between these image and suggest what may have caused these differences. </i> 

PROVIDE YOUR DESCRIPTION HERE!
</font>
</div>
<hr>

In [ ]:
# Step 4: Pick images from different years and display them side-by-side
# ENTER YOUR CODE HERE

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Step 5: </font>Describe the differences you see between these image and suggest what may have caused these differences.

PROVIDE YOUR DESCRIPTION HERE!
</font>
</div>
<hr>

<br>
<hr>
<font face="Calibri" size="5"> <b> 5. SAR Time Series Visualization, Animation, and Analysis </b> </font> 

<font face="Calibri" size="3"> This section introduces you to the handling and analysis of SAR time series stacks. A focus will be put on time series visualization, which allow us to inspect time series in more depth. Note that html animations are not exported into the pdf file, but will display interactively. </font>

In [ ]:
# Turn on inline presentations. This will make sure that your graphics will actually appear in the notebook 
# rather than a separate page.
%matplotlib inline

In [ ]:
# Let's import some additional Matplotlib Python libraries

import matplotlib.pyplot as plt
import matplotlib.patches as patches  # Needed to draw rectangles
from matplotlib import animation, rc
from IPython.display import HTML

<br>
<font face="Calibri" size="4"> <b> 5.1 Reading the SAR Time Series Subset </b> </font>

<font face="Calibri" size="3"> Let's read an image subset (offset 400, 400 /  size 600, 600) of the entire time series data stack. The data are linearly scaled amplitudes represented as unsigned 16 bit integer.

We use the gdal *ReadAsArray(xoff,yoff,xsize,ysize)* function where *xoff* is the offset in pixels from upper left; *yoff* is the offset in lines from upper left; *xsize* is the number of pixels and *ysize* is the number of lines of the subset.

If *ReadAsArray()* is called without any parameters, the entire image data stack is read. 

<b>Let's first define a subset and make sure it is in the right geographic location</b>. </font> 

In [ ]:
# Open the image and read the first raster band
band = img.GetRasterBand(1)

# Define the subset
subset=(400,400,600,600)
if subset==None:
    subset=(0,0,img.RasterXSize,img.RasterYSize)

In [ ]:
# Plot one band together with the outline of the selected subset to verify its geographic location.
raster=band.ReadAsArray()
vmin=np.percentile(raster.flatten(),5)
vmax=np.percentile(raster.flatten(),95)
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)
ax.imshow(raster,cmap='gray',vmin=vmin,vmax=vmax)
# plot the subset as rectangle
_=ax.add_patch(patches.Rectangle((subset[0],subset[1]),subset[2],subset[3],
                                 fill=False,edgecolor='red'))

<font face="Calibri" size="3"> Now we are ready to <b>extract this subset from all slices of the data stack</b>. </font>

In [ ]:
raster0 = band.ReadAsArray(*subset)
bandnbr=0 # Needed for updates
rasterstack=img.ReadAsArray(*subset)

<br>
<font face="Calibri" size="4"> <b> 5.2 Calibration and Data Conversion between dB and Power Scales </b> </font>

<font face="Calibri" size="3"> Focused SAR image data natively come in uncalibrated digital numbers (DN) and need to be calibrated to correspond to proper radar cross section information. 

Calibration coefficients for SAR data are often defined in the decibel (dB) scale due to the high dynamic range of the imaging system. For the L-band ALOS PALSAR data at hand, the conversion from uncalibrated DN values to calibrated radar cross section values in dB scale is performed by applying a standard **calibration factor of -83 dB**. 
<br> <br>
$\gamma^0_{dB} = 20 \cdot log10(DN) -83$

The data at hand are radiometrically terrain corrected images, which are often expressed as terrain flattened $\gamma^0$ backscattering coefficients. For forest and land cover monitoring applications $\gamma^o$ is the preferred metric.

<b>Let's apply the calibration constant for our data and export in *dB* scale</b>: </font> 

In [ ]:
caldB=20*np.log10(rasterstack)-83

<font face="Calibri" size="3"> While **dB**-scaled images are often "visually pleasing", they are often not a good basis for mathematical operations on data. For instance, when we compute the mean of observations, it makes a difference whether we do that in power or dB scale. Since dB scale is a logarithmic scale, we cannot simply average data in that scale. 
    
Please note that the **correct scale** in which operations need to be performed **is the power scale.** This is critical, e.g. when speckle filters are applied, spatial operations like block averaging are performed, or time series are analyzed.

To **convert from dB to power**, apply: $\gamma^o_{pwr} = 10^{\frac{\gamma^o_{dB}}{10}}$ </font>

In [ ]:
calPwr=np.power(10.,caldB/10.)

<br>
<font face="Calibri" size="4"> <b> 5.3 Create a Time Series Animation </b> </font>

<font face="Calibri" size="3"> Now we are ready to <b>create a time series animation</b> from the calibrated SAR data. </font> 

In [ ]:
%%capture 
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np

fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.axis('off')
vmin=np.percentile(caldB.flatten(),5)
vmax=np.percentile(caldB.flatten(),95)

r0dB=20*np.log10(raster0)-83

im = ax.imshow(r0dB,cmap='gray',vmin=vmin,vmax=vmax)
ax.set_title("{}".format(tindex[0].date()))

def animate(i):
    ax.set_title("{}".format(tindex[i].date()))
    im.set_data(caldB[i])

# Interval is given in milliseconds
ani = matplotlib.animation.FuncAnimation(fig, animate, 
                                         frames=rasterstack.shape[0],
                                        interval=400)

In [ ]:
rc('animation',embed_limit=40971520.0)  # We need to increase the 
            # limit maybe to show the entire animation

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

<br>
<font face="Calibri" size="4"> <b> 5.3 Plot the Time Series of Means Calculated Across the Subset </b> </font>

<font face="Calibri" size="3"> To create the time series of means, we will go through the following steps:
1. Ensure that you use the data in **power scale** ($\gamma^o_{pwr}$) for your mean calculations.
2. compute means.
3. convert the resulting mean values into dB scale for visualization.
4. plot time series of means. </font> 

In [ ]:
# 2. Compute Means using variable calPwr
rs_means_pwr = np.mean(calPwr,axis=(1,2))
# 3. Convert resulting mean value time-series to dB scale for visualization
rs_means_dB = 10.*np.log10(rs_means_pwr)

In [ ]:
rs_means_pwr.shape # Check that we got the means over time

In [ ]:
# 4. Now let's plot the time series of means
fig=plt.figure(figsize=(16,4))
ax1=fig.add_subplot(111)
ax1.plot(tindex,rs_means_pwr)
ax1.set_xlabel('Date')
ax1.set_ylabel('$\overline{\gamma^o}$ [power]')


ax2=ax1.twinx()
ax2.plot(tindex,rs_means_dB,color='red')
ax2.set_ylabel('$\overline{\gamma^o}$ [dB]')
fig.legend(['power','dB'],loc=1)
plt.title('Time series profile of average band backscatter $\gamma^o$ ')

<br>
<font face="Calibri" size="4"> <b> 5.4 Create Two-Panel Figure with Animated Global Mean $\mu_{\gamma^0_{dB}}$ </b> </font>

<font face="Calibri" size="3"> We use a few Matplotlib functions to create a side-by-side animation of the dB-scaled imagery and the respective global means $\mu_{\gamma^0_{dB}}$. </font> 

In [ ]:
%%capture 
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np

fig, (ax1,ax2) = plt.subplots(1,2,figsize=(16,4),gridspec_kw = {'width_ratios':[1, 3]})

vmin=np.percentile(rasterstack.flatten(),5)
vmax=np.percentile(rasterstack.flatten(),95)
im = ax1.imshow(raster0,cmap='gray',vmin=vmin,vmax=vmax)
ax1.set_title("{}".format(tindex[0].date()))
ax1.set_axis_off()

ax2.axis([tindex[0],tindex[-1],rs_means_dB.min(),rs_means_dB.max()])
ax2.set_ylabel('$\overline{\gamma^o}$ [dB]')
ax2.set_xlabel('Date')
ax2.set_ylim((-10,-5))
l, = ax2.plot([],[])


def animate(i):
    ax1.set_title("{}".format(tindex[i].date()))
    im.set_data(rasterstack[i])
    ax2.set_title("{}".format(tindex[i].date()))
    l.set_data(tindex[:(i+1)],rs_means_dB[:(i+1)])

# Interval is given in milliseconds
ani = matplotlib.animation.FuncAnimation(fig, animate, 
                                         frames=rasterstack.shape[0],
                                        interval=400)

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

<br>
<hr>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #3</u>:  </font> Discuss the Global Mean Time-Series Plot </b> <font color='rgba(200,0,0,0.2)'> -- [4 Points] </font> </font>

<font face="Calibri" size="3"> <u>For this exercise please answer the following questions</u>:
<ol type="1">
    <li>What kind of temporal patterns do you see in the time series for this subset. <font color='rgba(200,0,0,0.2)'> -- [2 Points] </font></li>
    <br>
    <li>What may explain the temporal patterns you observed? Relate your answer to Section 1 of this Notebook and to the information about what drives radar brighness from the class lectures.<font color='rgba(200,0,0,0.2)'> -- [2 Point] </font></li> 
</ol> 
</font>
</div>

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Step 1: </font> What temporal patterns do you see in the global mean time series for your subset? </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<div class="alert alert-success"><font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Step 2: </font> What may explain the temporal patterns you observed? </i> 

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>ASSIGNMENT #4</u>:  </font> Select a Different (non-overlapping) Subset and Redo Analysis</b> <font color='rgba(200,0,0,0.2)'> -- [6 Points] </font> </font>

<font face="Calibri" size="3"> <u>For this exercise please complete the following tasks</u>:
<ol type="1">
    <li>Define a second subset stack. <font color='rgba(200,0,0,0.2)'> -- [1 Point] </font></li>
    <br>
    <li>Compute the means (use power data for this step)<font color='rgba(200,0,0,0.2)'> -- [1 Points] </font></li>
    <br>
    <li>Convert to dB for visualization<font color='rgba(200,0,0,0.2)'> -- [1 Points] </font></li>
    <br>
    <li>Create two panel figure with Animated Global Mean $\mu_{\gamma^0_{dB}}$<font color='rgba(200,0,0,0.2)'> -- [1 Point]
    </font></li>
    <br>
    <li>Discuss the time series plot. Discuss differences to the previous time series should there be any. <font color='rgba(200,0,0,0.2)'> -- [2 Point]
    </font></li>
</ol> 
</font>
</div>
<hr>

In [ ]:
# Step 1: Extract second (non-overlapping) subset
# ENTER YOUR CODE HERE

<hr>

In [ ]:
# Steps 2 - 3: Compute means and convert to dB
# ENTER YOUR CODE HERE

<hr>

In [ ]:
# Step 4: Create two panel figure with animated global means
# ENTER YOUR CODE HERE

<hr>
<div class="alert alert-success">
<font face="Calibri" size="3"> <i><font color='rgba(200,0,0,0.2)'> Step 5: </font> Discuss time series plot.

PROVIDE YOUR ANSWER HERE!
</font>
</div>
<hr>

<font face="Calibri" size="2"> <i>GEOS 657 Microwave Remote Sensing - Version 1.0 - Feb 2019 </i>
</font>